In [12]:
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation
import numpy as np

# Load the data

[link](https://www.kaggle.com/c/sf-crime/data?test.csv.zip)

In [13]:
#Load Data with pandas, and parse the first column into datetime
train_csv = r'data/train.csv'
%time train=pd.read_csv(train_csv, parse_dates = ['Dates'])

test_csv = r'data/test.csv'
%time test=pd.read_csv(test_csv, parse_dates = ['Dates'])

CPU times: user 3.05 s, sys: 222 ms, total: 3.28 s
Wall time: 3.31 s
CPU times: user 2.71 s, sys: 203 ms, total: 2.91 s
Wall time: 2.94 s


# Encode labels

In [14]:
labelEncoder = preprocessing.LabelEncoder()
encodedCategory = labelEncoder.fit_transform(train.Category)

encodedDays = pd.get_dummies(train.DayOfWeek)
encodedDistrict = pd.get_dummies(train.PdDistrict)
trainDf = pd.concat([encodedDays, encodedDistrict], axis=1)

trainDf['Y']=train['Y']
trainDf['X']=train['X']
trainDf['Category']=encodedCategory

encodedDays = pd.get_dummies(test.DayOfWeek)
encodedDistrict = pd.get_dummies(test.PdDistrict)
testDf = pd.concat([encodedDays, encodedDistrict], axis=1)

testDf['Y']=test['Y']
testDf['X']=test['X']

# Logistic Regression Classifier

In [15]:
features = ['Y', 'X']
features = np.append(features, train.DayOfWeek.unique())
features = np.append(features, train.PdDistrict.unique())
features.size

19

In [16]:
def logisticRegressionClassifier (df, size, features):
    # Split arrays or matrices into random train and test subsets
    training, validation = train_test_split(df, train_size=size)

    model = LogisticRegression()
    model.fit(training[features], training['Category'])
    predicted = model.predict_proba(validation[features])

    print('(size, log_loss) : ',(size,log_loss(validation['Category'], np.array(predicted))))

In [8]:
%time logisticRegressionClassifier(trainDf,0.7,features)

(size, log_loss) :  (0.7, 2.6118485124038018)
CPU times: user 2min 35s, sys: 3.9 s, total: 2min 39s
Wall time: 2min 52s


# Cross Validation with logistic regression

In [17]:
def crossValidation (df, size ,features, cvGen=3):
    # Split arrays or matrices into random train and test subsets
    training, validation = train_test_split(df, train_size=size)

    model = LogisticRegression()
    log_loss = cross_validation.cross_val_score(model,training[features],training['Category'],cv=cvGen,scoring='log_loss')
    
    print('(size, cv,  log_loss) : ',(size, cvGen, log_loss))

In [18]:
%time crossValidation(trainDf,0.7,features)  #cv = 3
%time crossValidation(trainDf,0.7,features,4)

(size, cv,  log_loss) :  (0.7, 3, array([-2.61355628, -2.61416839, -2.61262285]))
CPU times: user 5min 10s, sys: 6.82 s, total: 5min 17s
Wall time: 5min 20s
(size, cv,  log_loss) :  (0.7, 4, array([-2.6132108 , -2.61325661, -2.61452945, -2.61337517]))
CPU times: user 7min 57s, sys: 7.43 s, total: 8min 5s
Wall time: 8min 14s
